In [1]:
!wget https://sites.google.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attredirects=0&d=1 
!unzip /content/OLIDv1.0.zip?attredirects=0

--2021-03-29 04:09:22--  https://sites.google.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attredirects=0
Resolving sites.google.com (sites.google.com)... 142.250.99.102, 142.250.99.113, 142.250.99.138, ...
Connecting to sites.google.com (sites.google.com)|142.250.99.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://ef80a887-a-62cb3a1a-s-sites.googlegroups.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attachauth=ANoY7cpN_f9SN4nPK-FD9a_9jaVL1uCqyTpmEJntYeqYIRAtNx9loxUrGbIPUoks09MYwBrPUAwhOQk_x8jYB6SwR8NiN-J7NUCUbfrdaKaaKs6HikN3UJiSorYdbkhsI-VqpuFG33o9lUI5RU-rwr5WaSBIZKZi4Wo50zay7o_tx-Qm4HXTtge9dfq6_1Nl8RWB40RokCW6N_fPZ2KBaWJtejq8Gkn-bt9ZNRSDQXco61tdqLsbFkI%3D&attredirects=0 [following]
--2021-03-29 04:09:22--  https://ef80a887-a-62cb3a1a-s-sites.googlegroups.com/site/offensevalsharedtask/olid/OLIDv1.0.zip?attachauth=ANoY7cpN_f9SN4nPK-FD9a_9jaVL1uCqyTpmEJntYeqYIRAtNx9loxUrGbIPUoks09MYwBrPUAwhOQk_x8jYB6SwR8NiN-J7NUCUbfrdaKaaKs6HikN

In [2]:
!pip install transformers
!pip install datasets
# !pip install sentencepiece

     |████████████████████████████████| 2.0MB 7.5MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
     |████████████████████████████████| 3.2MB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=42196f5824e93765d31aa1853e0bfc6d8f18c612dda8a19ae53da10e90774a0c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 8.7MB/s 
     |████████████████████████████████| 245kB 35.2MB/s 
     |████████████████████████████████| 112kB 34.2MB/s 


In [50]:
import pandas as pd
import numpy as np
import statistics

import torch 
import torch.nn.functional as F
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from tqdm.notebook import tqdm

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [58]:
df = pd.read_csv("/content/olid-training-v1.0.tsv", sep='\t')
df_test = pd.read_csv("/content/testset-levela.tsv", sep='\t')
df_test = pd.merge(df_test, pd.read_csv("/content/labels-levela.csv", names=['id','subtask_a' ]), on='id')

df = df[['tweet','subtask_a']]
df_test =  df_test[['tweet','subtask_a']]
labels = {'NOT':0, 'OFF':1}
df['subtask_a'] = [(float(labels[label])) for label in df.subtask_a]
df_test['subtask_a'] = [(float(labels[label])) for label in df_test.subtask_a]

remove_list = ['@USER', 'URL']
remove_words = lambda x: ' '.join([word for word in x.split() if word not in remove_list])

df['tweet'] = df['tweet'].apply(remove_words)
df_test['tweet'] = df_test['tweet'].apply(remove_words)

In [6]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 5
LEARNING_RATE = 2e-5

MODEL_NAME_OR_PATH = 'bert-base-uncased'

In [60]:
train = df
test = df_test
x_train, y_train = train['tweet'].values.tolist(), train['subtask_a'].values.tolist()
x_test, y_test = test['tweet'].values.tolist(), test['subtask_a'].values.tolist()

print(train.shape)
print(test.shape)

(13240, 2)
(860, 2)


In [61]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config =  BertConfig.from_pretrained(MODEL_NAME_OR_PATH)
config.num_labels = 1

model = BertForSequenceClassification.from_pretrained(MODEL_NAME_OR_PATH, config= config).to(device)
model.train()
train_encodings = tokenizer(x_train, truncation=True, padding=True,  return_tensors='pt')
test_encodings = tokenizer(x_test, truncation=True, padding=True, return_tensors='pt')
print(train_encodings)
print(config.to_json_string())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'input_ids': tensor([[  101,  2016,  2323,  ...,     0,     0,     0],
        [  101,  2175,  2188,  ...,     0,     0,     0],
        [  101,  9733,  2003,  ...,     0,     0,     0],
        ...,
        [  101,  1998,  2339,  ...,     0,     0,     0],
        [  101, 22418,   102,  ...,     0,     0,     0],
        [  101,  1001,  3009,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpoi

In [62]:
class OLID_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self, idx):

    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

def create_data_loader(x, y, batch_size):
    dataset = OLID_Dataset(x,y)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

train_dataloader = create_data_loader(train_encodings, y_train, TRAIN_BATCH_SIZE)
test_dataloader = create_data_loader(test_encodings, y_test, TEST_BATCH_SIZE)


In [63]:
def eval_op():
  y_true = []
  y_pred = []
  with torch.no_grad():
    for dl in tqdm(test_dataloader, total=len(test_dataloader), desc="Training... "):
      input_ids = dl['input_ids'].to(device)
      attention_mask = dl['attention_mask'].to(device)
      targets = dl['labels'].to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      y_true += targets.cpu().numpy().tolist()
      y_pred += nn.Sigmoid()(outputs.logits).round().squeeze().cpu().tolist()
    print('ACCURACY:',accuracy_score(y_true,y_pred))


In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, 0, 5 * 745)

for epoch in range(5):
  step = 0 
  losses = []
  corrects = 0
  for dl in tqdm(train_dataloader, total=len(train_dataloader), desc="Training... "):
    optimizer.zero_grad()
    step += 1
    input_ids = dl['input_ids'].to(device)
    attention_mask = dl['attention_mask'].to(device)
    targets = dl['labels'].to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    loss = loss_fn(outputs.logits, targets.unsqueeze(1))
    loss.backward()
    optimizer.step()
    scheduler.step()
    corrects += (nn.Sigmoid()(outputs.logits).round().squeeze().detach().cpu() == targets.cpu()).sum()
    losses.append(loss.detach().cpu().item())
    if step%100 == 0:
      print(statistics.mean(losses), corrects/(step*16))
  eval_op()
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


0.6245236849784851 tensor(0.6625)
0.5642003829777241 tensor(0.7131)
0.5273687799274921 tensor(0.7410)
0.5097867787629365 tensor(0.7534)
0.4952535735964775 tensor(0.7639)
0.4825987236822645 tensor(0.7715)
0.4752833631315402 tensor(0.7758)
0.46898548477329316 tensor(0.7795)




ACCURACY: 0.8418604651162791


0.418467128276825 tensor(0.8087)
0.38965419702231885 tensor(0.8281)
0.3701438759764036 tensor(0.8421)
0.3633309457823634 tensor(0.8438)
0.35520588791370394 tensor(0.8486)
0.3507132922237118 tensor(0.8520)
0.34703586448516166 tensor(0.8516)


In [51]:
eval_op()
precision, recall, fscore, support = score(y_true, y_pred)
print('PERCESION: {}'.format(precision))
print('RECALL: {}'.format(recall))
print('FSCORE: {}'.format(fscore))
print('SUPPORT: {}'.format(support))
print('F1 SCORE: {}'.format(f1_score(y_true, y_pred)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  



ACCURACY: 0.7953172205438066
PERCESION: [0.83949314 0.76392573]
RECALL: [0.89932127 0.65454545]
FSCORE: [0.86837794 0.70501836]
SUPPORT: [884 440]
F1 SCORE: 0.7050183598531212


precision: [0.83949314 0.76392573]
recall: [0.89932127 0.65454545]
fscore: [0.86837794 0.70501836]
support: [884 440]
